## Linear Regression Project in Spark

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lin_project').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv('../datasets/Ecommerce_Customers.csv',inferSchema=True,header=True)

In [7]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [9]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [10]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [11]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

`Transforming all numerical features columns to a vector, that's how spark expect it`

In [12]:
assembler = VectorAssembler(inputCols=[ 'Avg Session Length','Time on App','Time on Website','Length of Membership'],
                            outputCol='features')

In [13]:
output = assembler.transform(data)

In [14]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [16]:
# Let's see our new features column
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

#### Let's select our features and label column

In [17]:
final_data = output.select('features','Yearly Amount Spent')

In [18]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

### Train Test Split

In [19]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [21]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                350|
|   mean|  503.2496744259856|
| stddev|  75.27620687115811|
|    min| 256.67058229005585|
|    max|  725.5848140556806|
+-------+-------------------+



In [22]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                150|
|   mean| 490.13088720133624|
| stddev|  87.60208334125143|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+



In [23]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [24]:
lr_model = lr.fit(train_data)

In [25]:
test_results = lr_model.evaluate(test_data)

In [26]:
test_results.rootMeanSquaredError

10.666347075329927

In [27]:
test_results.r2

0.9850752335497025